In [1]:
#기본 패키지
import time
import warnings
warnings.filterwarnings('ignore')

#데이터 처리 패키지
import numpy as np
import pandas as pd

#시각화 패키지
import matplotlib as mpl
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
plt.rc('font', family=fm.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()) # for Windows OS user
from matplotlib import rc 
rc('font', family='malgun gothic')
rc('axes', unicode_minus = False)
import seaborn as sns
#plt.style.use('ggplot')

#수학 패키지
from scipy.stats.mstats import gmean
from scipy.stats import randint as sp_randint
from scipy.stats import uniform
import uuid
import random
from tqdm import tqdm, tqdm_notebook
from itertools import combinations

#머신러닝 패키지
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import ShuffleSplit
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler,RobustScaler
from sklearn.feature_selection import SelectPercentile
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import *
from sklearn.naive_bayes import GaussianNB 
from sklearn.linear_model import LogisticRegression, SGDClassifier, LinearRegression, SGDRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.svm import SVC
from xgboost import XGBClassifier, XGBRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.cross_decomposition import PLSRegression
from sklearn.linear_model import Lasso,ElasticNet,Ridge
from sklearn.linear_model import BayesianRidge
from vecstack import stacking, StackingTransformer
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error as mse

#딥러닝 패키지
import tensorflow.keras as keras
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import * #Model
from tensorflow.keras.optimizers import *
from tensorflow.keras.regularizers import *
#from keras.utils.np_utils import *
#from tensorflow.keras.utils.vis_utils import * #model_to_dot
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras.losses import *
from tensorflow.keras import layers
print('tf Version \n', tf.__version__)
print('tf.keras Version \n', tf.keras.__version__)

import kerastuner as kt
import datetime
import os
import sys, warnings
if not sys.warnoptions: warnings.simplefilter("ignore")

import shap

import klib
# Dacon plotly 그림 업로드 
pd.options.plotting.backend = 'plotly'
## plotly.io를 import 한 후 renderers 기본값을 꼭 "notebook_connected" 로 설정해주시기 바랍니다.
import plotly.io as pio
pio.renderers.default = "notebook_connected"

import optuna
from optuna import Trial
from optuna.samplers import TPESampler

tf Version 
 2.3.0
tf.keras Version 
 2.4.0


In [11]:
cus = pd.read_csv("open/cus_info.csv")
hist = pd.read_csv("open/stk_bnc_hist.csv")
iem = pd.read_csv("open/iem_info.csv")
train = pd.read_csv("open/stk_hld_train.csv")
test = pd.read_csv("open/stk_hld_test.csv")

submission = pd.read_csv("open/sample_submission.csv")

In [12]:
cus.columns=['계좌번호','성별','연령대','투자성향','자산구간','주거래상품군','Life_style','서비스등급','총투자기간','주거래업종구분']
hist.columns=['계좌번호','기준일자','종목코드','잔고수량','잔고금액','주당액면가']
iem.columns=['종목코드','종목한글명','종목업종','시가총액규모유형','시장구분']
train.columns=['계좌번호','종목코드','매수일자','hold_d']
test.columns=['계좌번호','종목코드','매수일자','과거보유일','submit_id','hold_d']

In [13]:
hist['기준일자']=pd.to_datetime(hist.기준일자.astype(str))
train['매수일자']=pd.to_datetime(train.매수일자.astype(str))
test['매수일자']=pd.to_datetime(test.매수일자.astype(str))

과거보유일을 만들기 위해서는 test에 일관적 기준인 2020년 12월 30일처럼 train 행별로 기준을 만들어준다면 과거보유일을 쉽게 만들 수 있다.\
그 기준을 어떻게 만드는지가 중요한 포인트인 것 같다.

1. 특정 고객이 어느년도에 구매했는지에 따라 기준이 생긴다?
2. 특정 고객이 어느 달에 구매했는지에 따라 기준이 생긴다?
3. 특정 고객이 어떤 종목을 구매했는지에 따라 기준이 생긴다?

cus 따로 iem따로 hist따로 열을 만든 다음 더해주는 것이 좋은가?
어떤 것이 키포인트인지 생각해보자

In [ ]:
cus['연령대']=cus.연령대.apply(lambda x: 20 if x ==1 else 25 if x==2 else 30 if x==3 else 35 if x==4 else 40 if x==5 else 45 if x==6 
              else 50 if x==7 else 55 if x==8 else 60)

In [ ]:
cus['자산구간']=cus.자산구간.apply(lambda x: 0 if x ==1 else 10000000 if x==2 else 30000000 if x==3 
                           else 50000000 if x==4 else 100000000 if x==5 else 300000000 )

In [7]:
cus['총투자기간']=cus.총투자기간.apply(lambda x:  0 if x ==1 else 128 if x==2 else 260 if x==3 
                           else 782 if x==4 else 1303 if x==5 else 2607 )

In [8]:
# 계좌번호 자체를 보여줄 수 있는 필력할 수 있는 그런 열을 만들면 좋을 것으로 생각된다.
#워드투백을 하겠다는 것
cus.Life_style.astype(str) +'_' + cus.서비스등급.astype(str) +'_' + cus.주거래상품군.astype(str)  +'_'+ cus.투자성향.astype(str) # + cus.총투자기간 ???

0       3_3_2_99
1        5_2_2_4
2        5_5_2_4
3        5_3_8_4
4        5_5_2_2
          ...   
9995     9_5_2_4
9996     2_9_2_3
9997     5_5_2_4
9998    5_3_2_99
9999     5_3_2_0
Length: 10000, dtype: object

In [9]:
cus.주거래업종구분.value_counts()
# 고객의 주거래업종과 그 주식의 종목업종이 같은지를 보고 처리해주기

10    2107
16    2024
9     1320
13     867
5      831
8      550
14     496
6      488
12     325
7      324
2      322
11      83
3       79
1       75
15      63
4       46
Name: 주거래업종구분, dtype: int64

#### 종목 별 주당액면가에 관련된 피처

- 주당액면가가 변한 주식도 있다. 이 차이를 넣어주면 좋을 것 같다.
- 주당액면가의  max-min
- 주당액면가의 nunique도 넣어줘도 될꺼같다? 크게 의미는 없겠지만?

In [14]:
hist["평균잔고금액"] = hist["잔고금액"] / hist["잔고수량"]
#hist = hist.fillna(0)

In [15]:
액면가_변동폭=hist.groupby('종목코드')['주당액면가'].agg([('액면가_변동폭', lambda x: np.max(x)-np.min(x))]).reset_index()
액면가_변동수=hist.groupby('종목코드')['주당액면가'].nunique().reset_index()

액면가_변동폭['액면가_변동수']=액면가_변동수['주당액면가']

In [16]:
hist=pd.merge(hist,액면가_변동폭,how='left',on='종목코드')

#### 잔고수량 & 잔고금액을 이용한 피쳐

In [17]:
hist[(hist.계좌번호==hist.계좌번호[0])&(hist.종목코드==hist.종목코드[0])]

,계좌번호,기준일자,종목코드,잔고수량,잔고금액,주당액면가,평균잔고금액,액면가_변동폭,액면가_변동수
0,1119c23c3a504ca7b75060277410c0f6fb9018ec7638c2...,2020-08-20,A008770,40.0,2828000.0,5000.0,70700.0,0.0,1
1,1119c23c3a504ca7b75060277410c0f6fb9018ec7638c2...,2020-06-23,A008770,20.0,1390000.0,5000.0,69500.0,0.0,1


In [18]:
test[(test.계좌번호==hist.계좌번호[0])&(test.종목코드==hist.종목코드[0])]

,계좌번호,종목코드,매수일자,과거보유일,submit_id,hold_d
38,1119c23c3a504ca7b75060277410c0f6fb9018ec7638c2...,A008770,2020-06-23,131,IDX00039,0


- hist를 이용해서 이 고객이 이 종목을 몇번 구매했거나 판매했는지 그리고 잔고수량의 변동이 어땠는지 그리고 그 때마다 평균금액이 올랐는지 내렸는지를 표현할 수 있을 것 같다.

In [19]:
# diff 를 하기 위해 순서대로 정렬해준 것
hist_order=hist.sort_values(by=['계좌번호','종목코드','기준일자'],ascending=True).reset_index(drop='index')

In [20]:
hist_order[(hist_order.계좌번호==hist_order.계좌번호[2573836])&(hist_order.종목코드==hist_order.종목코드[2573836])]

,계좌번호,기준일자,종목코드,잔고수량,잔고금액,주당액면가,평균잔고금액,액면가_변동폭,액면가_변동수
2573832,fff9cf3aaffd32dcbf70dce8e5216411f9d38537b477d6...,2020-09-14,A293490,2.0,147600.0,100.0,73800.0,0.0,1
2573833,fff9cf3aaffd32dcbf70dce8e5216411f9d38537b477d6...,2020-09-16,A293490,3.0,198900.0,100.0,66300.0,0.0,1
2573834,fff9cf3aaffd32dcbf70dce8e5216411f9d38537b477d6...,2020-10-13,A293490,4.0,196600.0,100.0,49150.0,0.0,1
2573835,fff9cf3aaffd32dcbf70dce8e5216411f9d38537b477d6...,2020-11-24,A293490,5.0,245500.0,100.0,49100.0,0.0,1
2573836,fff9cf3aaffd32dcbf70dce8e5216411f9d38537b477d6...,2020-12-14,A293490,0.0,0.0,100.0,NaN,0.0,1


In [21]:
평균잔고금액_na=hist_order[hist_order['평균잔고금액'].isnull()==1].index

 고객이 그 종목을 새로 시도한 횟수(0에서 시작한 횟수)

In [23]:
train_t=train.copy()
train_t['시작날짜']=1

In [25]:
test_t=test.copy()
test_t['시작날짜']=1
test_t=test_t[['계좌번호','종목코드','매수일자','hold_d','시작날짜']]

In [32]:
tt=pd.concat((train_t,test_t)).reset_index(drop=True)

In [33]:
hist_order=pd.merge(hist_order,tt[['계좌번호','종목코드','매수일자','시작날짜']],how='left',left_on=['계좌번호','종목코드','기준일자'],right_on=['계좌번호','종목코드','매수일자'])
hist_order.drop('매수일자',axis=1,inplace=True)

In [35]:
aa=hist_order.groupby(['계좌번호','종목코드']).시작날짜.count().reset_index()
aa.columns=['계좌번호', '종목코드', '시작횟수']

In [37]:
hist_order=pd.merge(hist_order, aa,how='left',on=['계좌번호','종목코드'])

고객이 특정 종목을 여러번 시작했을 경우 밑의 전처리에서 계좌번호와 종목코드로만 묶을 경우 해소되지 않는 부분을 해소하기 위한 방법

In [43]:
tt=tt.reset_index()
tt.columns=['종류별', '계좌번호', '종목코드', '매수일자', 'hold_d', '시작날짜']
tt.종류별=tt.종류별+1

In [44]:
hist_order=pd.merge(hist_order,tt[['계좌번호','종목코드','매수일자','종류별']],how='left',left_on=['계좌번호','종목코드','기준일자'],right_on=['계좌번호','종목코드','매수일자'])
hist_order.drop('매수일자',axis=1,inplace=True)

In [45]:
hist_order['종류별']=hist_order.종류별.fillna(method='pad')
hist=pd.merge(hist,hist_order[['계좌번호','기준일자','종목코드','종류별']],how='left',on=['계좌번호','기준일자','종목코드'])

In [50]:
잔고_수량_차분=hist_order[['기준일자','종류별','계좌번호','종목코드']]
# 먼저 
잔고_수량_차분['잔고수량_차분']=hist_order.groupby('종류별')['잔고수량'].diff()

잔고수량 차분 관련해서는 평균은 넣으면 안되고(팔렸으면 평균 0이기 때문에) 편차는 넣어도 될 것 같다.
최대와 최소는 넣기 애매하다. \
이것을 고객별로 묶어서 해야할까 아님 고객별로 종모코드별로 묶어야할까?

In [51]:
#결측치 넣기
잔고_수량_차분.loc[잔고_수량_차분['잔고수량_차분'].isna(), '잔고수량_차분'] = hist_order.loc[잔고_수량_차분['잔고수량_차분'].isna(), '잔고수량']

In [52]:
#잔고_수량_차분['기준일자']=pd.to_datetime(잔고_수량_차분.기준일자.astype(str))
잔고_수량_차분['년']=잔고_수량_차분.기준일자.dt.year
# 밑에 그룹바이 할때 년도도 넣을 것인지 고민하기

In [53]:
잔고_수량_차분_1=잔고_수량_차분.groupby('종류별')['잔고수량_차분'].agg([('잔고수량_편차', np.std),
                                                                         ('잔고수량_최대', np.max),
                                                                         ('잔고수량_최소', np.min)]).reset_index()

In [54]:
년_잔고_수량_차분=잔고_수량_차분.groupby(['종류별','년'])['잔고수량_차분'].agg([('년_잔고수량_편차', np.std),
                                                                         ('년_잔고수량_최대', np.max),
                                                                         ('년_잔고수량_최소', np.min)]).reset_index()

In [55]:
hist=pd.merge(hist,잔고_수량_차분_1,how='left',on='종류별')

In [56]:
hist['년']=hist.기준일자.dt.year
hist=pd.merge(hist,년_잔고_수량_차분,how='left',on=['종류별','년'])

---

In [57]:
잔고_금액_차분=hist_order.drop(index=평균잔고금액_na,axis=0)

In [80]:
# 그 다음 평균잔고금액에 0인 친구들(다 팔아서) 그 행을 제외하고 나머지에 대한 diff를 보기 위해 제외시키고 diff를 진행한다.
잔고_금액_차분['잔고금액_차분']=잔고_금액_차분.groupby('종류별')['평균잔고금액'].diff()

In [81]:
잔고_금액_차분.fillna(0,inplace=True)

In [82]:
잔고_금액_차분_1=잔고_금액_차분.groupby('종류별')['잔고금액_차분'].agg([('잔고금액_평균', np.mean),
                                                                         ('잔고금액_최대', np.max),
                                                                         ('잔고금액_최소', np.min)]).reset_index()

In [83]:
잔고_금액_차분_1['잔고금액_극값']=잔고_금액_차분_1.잔고금액_최대+잔고_금액_차분_1.잔고금액_최소

In [84]:
잔고_금액_차분_1.drop(['잔고금액_최대','잔고금액_최소'],axis=1,inplace=True)

In [85]:
# 평균잔고금액의 last - first 를 하면 이 주식 상태가 좋아졌는지 안좋아졌는지를 표현할 수 있다.
잔고금액_변동=hist_order.drop(index=평균잔고금액_na,axis=0)
잔고금액_last=잔고금액_변동.groupby('종류별')['평균잔고금액'].last().reset_index()
잔고금액_first=잔고금액_변동.groupby('종류별')['평균잔고금액'].first().reset_index()
잔고금액_last['잔고금액_변동']=잔고금액_last.평균잔고금액 - 잔고금액_first.평균잔고금액

In [86]:
# 잔고수량으로 그냥 사요한 것
hist_잔고수량=hist.groupby('종류별')['잔고수량'].agg([ ('종목_거래횟수',np.size),
                                                        ('수량_최대',np.max),
                                                        ('수량_편차',np.std)]).reset_index()

In [87]:
hist=pd.merge(hist, 잔고_금액_차분_1,how='left',on='종류별')
hist=pd.merge(hist, 잔고금액_last[['종류별','잔고금액_변동']], how='left',on='종류별')
hist=pd.merge(hist, hist_잔고수량, how='left',on='종류별')

---

투자성향에 따른 보유기간도 다르다고 생각한다.\
투자성향 + 종목업종에 따른 보유기간이 좋을까나..

hold_d에 0.6을 곱한 값을 과거보유일로 했을 때 영향력이 아주 강했다.\
타겟값을 그대로 사용하였을 때 이러한 결과가 나왔기 때문에 카테고리별로 이러한 값을 넣어주어도 과적합이 일어나지 않을 것으로 보인다.

- 우리가 예측해야하는 값이 뭐냐? - 바로 2020년 12월 31일 이후 살아있는 주식들이 얼마나 보유될 것인가를 예측해야한다.

지금 우리에게 있는 데이터는 train 과 test 데이터가 있다.\
train 데이터에는 16년부터 20년 12월30일까지 종료된, 한마디로 팔아버린 주식에 대한 데이터가 있다.\
test 에는 20년 12월30일 이전 구매한 주식을 아직 가지고 있는 데이터가 있다.\
train에는 16년에 사고 16년에 판 주식도 있을 것이고 16년에 사고 20년 12월에 판 주식도 있을 것이다.

### 과거보유일을 어떻게 넣어줄 것인가가 중요한 것이다!!!

16 년 12월 30일을 기준으로 잘라서 데이터 생성(과거보유일, hold_d 가능)\
17 년 12월 30일을 기준으로 잘라서 데이터 생성(과거보유일, hold_d 가능)\
18 년 12월 30일을 기준으로 잘라서 데이터 생성(과거보유일, hold_d 가능)\
19 년 12월 30일을 기준으로 잘라서 데이터 생성(과거보유일, hold_d 가능)

20 년 12월 30일을 기준으로 잘라서 데이터 --> test_data

markov chain 의 아이디어

주식이 bull market, bear market, stagement market 각각에 있을 확률과 변화할 확률이 계속적으로 바뀔것이기 때문에 그에 따른 보유기간이 가능할까?

1. 계좌번호와 종목코드별 아이디를 만들어주는 것

2. 고객아이디를 cus.Life_style + cus.서비스등급 + cus.주거래상품군 + cus.투자성향 + cus.총투자기간 ???

3. hist data 를 좀 사용해볼 것

In [88]:
train["과거보유일"] = train["hold_d"]*0.6
train.과거보유일 = np.trunc(train["과거보유일"])

In [89]:
train_data = pd.merge(train, cus, how = "left", on = ["계좌번호"])
train_data = pd.merge(train_data, iem, how = "left", on = ["종목코드"])

test_data = pd.merge(test, cus, how = "left", on = ["계좌번호"])
test_data = pd.merge(test_data, iem, how = "left", on = ["종목코드"])

In [90]:
train_data['년']=train_data.매수일자.dt.year
train_data['월']=train_data.매수일자.dt.month
train_data['일']=train_data.매수일자.dt.day
train_data['요일']=train_data.매수일자.dt.day_of_week

test_data['년']=test_data.매수일자.dt.year
test_data['월']=test_data.매수일자.dt.month
test_data['일']=test_data.매수일자.dt.day
test_data['요일']=test_data.매수일자.dt.day_of_week

In [91]:
hist.drop('년',axis=1,inplace=True)
hist.fillna(0,inplace=True)

In [92]:
train_data = pd.merge(train_data, hist, how = "left", left_on = ["계좌번호", "종목코드",'매수일자'],right_on=['계좌번호','종목코드','기준일자'])
train_data.reset_index(drop = True, inplace = True)

test_data = pd.merge(test_data, hist, how = "left",  left_on= ["계좌번호", "종목코드",'매수일자'],right_on=['계좌번호','종목코드','기준일자'])
test_data.reset_index(drop = True, inplace = True)

In [93]:
train_test=pd.concat([train_data,test_data])
train_test=train_test.reset_index(drop='index')

In [94]:
# 년도별 잔고수량 평균 이상 이하
def f_2016(x):
    if x<20:
        return 0
    elif x<122:
        return 1
    elif x<400:
        return 2
    elif x<1247:
        return 3
    else:
        return 4
def f_2017(x):
    if x<8:
        return 0
    elif x<30:
        return 1
    elif x<100:
        return 2
    elif x<500:
        return 3
    else:
        return 4
def f_2018(x):
    if x<8:
        return 0
    elif x<30:
        return 1
    elif x<100:
        return 2
    elif x<400:
        return 3
    else:
        return 4
def f_2019(x):
    if x<6:
        return 0
    elif x<32:
        return 1
    elif x<110:
        return 2
    elif x<466:
        return 3
    else:
        return 4
def f_2020(x):
    if x<5:
        return 0
    elif x<25:
        return 1
    elif x<100:
        return 2
    elif x<300:
        return 3
    else:
        return 4

In [95]:
train_data['년도_잔고수량']=0
train_data.loc[train_data.년==2016,'년도_잔고수량']=train_data.query('년==2016')['잔고수량'].apply(f_2016)
train_data.loc[train_data.년==2017,'년도_잔고수량']=train_data.query('년==2017')['잔고수량'].apply(f_2017)
train_data.loc[train_data.년==2018,'년도_잔고수량']=train_data.query('년==2018')['잔고수량'].apply(f_2018)
train_data.loc[train_data.년==2019,'년도_잔고수량']=train_data.query('년==2019')['잔고수량'].apply(f_2019)
train_data.loc[train_data.년==2020,'년도_잔고수량']=train_data.query('년==2020')['잔고수량'].apply(f_2020)

test_data['년도_잔고수량']=0
test_data.loc[test_data.년==2016,'년도_잔고수량']=test_data.query('년==2016')['잔고수량'].apply(f_2016)
test_data.loc[test_data.년==2017,'년도_잔고수량']=test_data.query('년==2017')['잔고수량'].apply(f_2017)
test_data.loc[test_data.년==2018,'년도_잔고수량']=test_data.query('년==2018')['잔고수량'].apply(f_2018)
test_data.loc[test_data.년==2019,'년도_잔고수량']=test_data.query('년==2019')['잔고수량'].apply(f_2019)
test_data.loc[test_data.년==2020,'년도_잔고수량']=test_data.query('년==2020')['잔고수량'].apply(f_2020)

In [96]:
# 년도별 총투자기간 평균 이상 이하
def g_2016(x):
    if x>5.182650:
        return 1
    else:
        return 0
def g_2017(x):
    if x>4.162498:
        return 1
    else:
        return 0
def g_2018(x):
    if x>3.539667:
        return 1
    else:
        return 0
def g_2019(x):
    if x>3.324126:
        return 1
    else:
        return 0
def g_2020(x):
    if x>2.832396:
        return 1
    else:
        return 0

In [97]:
train_data['년도_총투자기간']=0
train_data.loc[train_data.년==2016,'년도_총투자기간']=train_data.query('년==2016')['총투자기간'].apply(g_2016)
train_data.loc[train_data.년==2017,'년도_총투자기간']=train_data.query('년==2017')['총투자기간'].apply(g_2017)
train_data.loc[train_data.년==2018,'년도_총투자기간']=train_data.query('년==2018')['총투자기간'].apply(g_2018)
train_data.loc[train_data.년==2019,'년도_총투자기간']=train_data.query('년==2019')['총투자기간'].apply(g_2019)
train_data.loc[train_data.년==2020,'년도_총투자기간']=train_data.query('년==2020')['총투자기간'].apply(g_2020)

test_data['년도_총투자기간']=0
test_data.loc[test_data.년==2016,'년도_총투자기간']=test_data.query('년==2016')['총투자기간'].apply(g_2016)
test_data.loc[test_data.년==2017,'년도_총투자기간']=test_data.query('년==2017')['총투자기간'].apply(g_2017)
test_data.loc[test_data.년==2018,'년도_총투자기간']=test_data.query('년==2018')['총투자기간'].apply(g_2018)
test_data.loc[test_data.년==2019,'년도_총투자기간']=test_data.query('년==2019')['총투자기간'].apply(g_2019)
test_data.loc[test_data.년==2020,'년도_총투자기간']=test_data.query('년==2020')['총투자기간'].apply(g_2020)

In [98]:
#
train_data['금액변화']=train_data.평균잔고금액-train_data.주당액면가
test_data['금액변화']=test_data.평균잔고금액-test_data.주당액면가


In [99]:
# 매수를 몇번 했는지
train_data=pd.merge(train_data,train_test.groupby('계좌번호')['년'].agg([('매수횟수',np.size)]).reset_index(),how='left',on='계좌번호')
test_data=pd.merge(test_data,train_test.groupby('계좌번호')['년'].agg([('매수횟수',np.size)]).reset_index(),how='left',on='계좌번호')

In [100]:
# 고객별 주 종목업종이 어떤것인지 파악을 한다.
고객_종목업종=train_test.groupby('계좌번호')['종목업종'].agg([('고객별_종목업종', lambda x: x.mode()[0])]).reset_index()

In [101]:
train_data=pd.merge(train_data,고객_종목업종,how='left',on='계좌번호')
test_data=pd.merge(test_data,고객_종목업종,how='left',on='계좌번호')

In [102]:
# 종목업종 별 hold_d 값의 평균을 넣어준 것
train_data=pd.merge(train_data,train_data.groupby('고객별_종목업종')['hold_d'].agg([('고객별_종목_hold',np.mean)]).reset_index(),how='left',on='고객별_종목업종')
test_data=pd.merge(test_data,train_data.groupby('고객별_종목업종')['hold_d'].agg([('고객별_종목_hold',np.mean)]).reset_index(),how='left',on='고객별_종목업종')

In [103]:
train_data['투자성형_자산구간']=train_data.투자성향*train_data.자산구간
test_data['투자성형_자산구간']=test_data.투자성향*test_data.자산구간

In [104]:
train_data['총투자제곱']=train_data.총투자기간**2
test_data['총투자제곱']=test_data.총투자기간**2

In [105]:
mean=train_data['hold_d'].mean()
agg=train_data.groupby('주거래업종구분')['hold_d'].agg(['count','mean'])
counts=agg['count']
means=agg['mean']
weight=80
smooth=(counts*means+weight*mean)/(counts+weight)

train_data['주거래업종구분_ME']=train_data['주거래업종구분'].map(smooth)
test_data['주거래업종구분_ME']=test_data['주거래업종구분'].map(smooth)

In [106]:
업종_주종목=train_test.groupby('종목업종')['종목한글명'].agg([('업종_주종목', lambda x: x.mode()[0])]).reset_index()

In [107]:
train_data=pd.merge(train_data, 업종_주종목, how='left', on='종목업종')
test_data=pd.merge(test_data, 업종_주종목, how='left', on='종목업종')

In [108]:
train_data.loc[train_data[(train_data["종목한글명"] == train_data["업종_주종목"])].index,'업종_주종목']=1
train_data['업종_주종목']=train_data.업종_주종목.apply(lambda x: x if x==1 else 0)

test_data.loc[test_data[(test_data["종목한글명"] == test_data["업종_주종목"])].index,'업종_주종목']=1
test_data['업종_주종목']=test_data.업종_주종목.apply(lambda x: x if x==1 else 0)

### word2vec

In [107]:
# 구매 정보 관련
train_data['개인정보'] = train_data.Life_style.astype(str) +'_' + train_data.서비스등급.astype(str) +'_' + train_data.주거래상품군.astype(str)  +'_'+ train_data.투자성향.astype(str) 
test_data['개인정보'] = test_data.Life_style.astype(str) +'_' + test_data.서비스등급.astype(str) +'_' + test_data.주거래상품군.astype(str)  +'_'+ test_data.투자성향.astype(str) 

In [108]:
import random

trn = list(train_data.groupby(['계좌번호','종목코드'])['개인정보'].unique())
tst = list(test_data.groupby(['계좌번호','종목코드'])['개인정보'].unique())

In [110]:
def oversample(x, n,seed=0):
    lst = []
    for i in x:
        tmp = []
        np.random.seed(seed)
        for j in range(n):
            random.shuffle(i)
            tmp += list(i)
            lst.append(tmp)
    return lst

In [148]:
w2v_input = oversample(trn+tst, 10)

In [149]:
from gensim.models import word2vec

w2v = word2vec.Word2Vec(sentences = w2v_input, size = 100, window = 5, min_count = 1, sg = 1,seed=0)

In [150]:
train_mean_vector = []
for words in tqdm(trn):
    tmp = np.zeros(100)
    cnt = 0
    for word in words:
        try:
            tmp += w2v[word]
            cnt += 1
        except:
            pass
    tmp /= cnt
    train_mean_vector.append(tmp)
train_mean_vector = np.array(train_mean_vector)

100%|███████████████████████████████████████████████████████████████████████| 433505/433505 [00:04<00:00, 96495.29it/s]


In [151]:
test_mean_vector = []
for words in tqdm(tst):
    tmp = np.zeros(100)
    cnt = 0
    for word in words:
        try:
            tmp += w2v[word]
            cnt += 1
        except:
            pass
    tmp /= cnt
    test_mean_vector.append(tmp)
test_mean_vector = np.array(test_mean_vector)

100%|█████████████████████████████████████████████████████████████████████████| 70596/70596 [00:00<00:00, 93112.58it/s]


In [152]:
train_mean_vector = pd.DataFrame(train_mean_vector)
test_mean_vector = pd.DataFrame(test_mean_vector)

In [153]:
train_mean_vector.isnull().sum().sum(),test_mean_vector.isnull().sum().sum()

(0, 0)

In [154]:
train_mean_vector.shape,test_mean_vector.shape

((433505, 100), (70596, 100))

In [161]:
aa=train_data.groupby(['계좌번호','종목코드'])['성별'].first().reset_index()
train_mean_vector[['계좌번호','종목코드']]=aa[['계좌번호','종목코드']]

bb=test_data.groupby(['계좌번호','종목코드'])['성별'].first().reset_index()
test_mean_vector[['계좌번호','종목코드']]=bb[['계좌번호','종목코드']]

In [167]:
train_data=pd.merge(train_data,train_mean_vector,how='left',on=['계좌번호','종목코드'])
test_data=pd.merge(test_data,test_mean_vector,how='left',on=['계좌번호','종목코드'])

In [109]:
train_data_select = train_data.drop(["계좌번호", "종목코드", "매수일자", "기준일자",'종류별'], axis = 1)
test_data_select = test_data.drop(["계좌번호", "종목코드", "매수일자", "submit_id", "hold_d", "기준일자",'종류별'], axis = 1)

In [110]:
# train_data에서 Y값을 추출한 후 hold_d column을 지워주겠습니다.

train_label = train_data_select["hold_d"]
train_data_select.drop(["hold_d"], axis = 1, inplace = True)

In [111]:
L_encoder = LabelEncoder()
L_encoder.fit(iem["종목한글명"])
train_data_select["종목한글명"] = L_encoder.transform(train_data_select["종목한글명"])
test_data_select["종목한글명"] = L_encoder.transform(test_data_select["종목한글명"])

In [112]:
#a=pd.get_dummies(train_data['투자_자산_wv'])
#b=pd.get_dummies(test_data['투자_자산_wv'])

In [113]:
#a_sub_b = [x for x in a.columns if x not in b.columns]
#a.drop(a_sub_b,axis=1,inplace=True)

In [114]:
#b_sub_a = [x for x in b.columns if x not in a.columns]
#b.drop(b_sub_a,axis=1,inplace=True)

In [115]:
#train_data=pd.concat([train_data,a],axis=1)
#test_data=pd.concat([test_data,b],axis=1)

In [116]:
train_data_select.shape,test_data_select.shape

((681472, 46), (70596, 46))

In [117]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

#테스트 스케일링은 data leakge 방지를 위해 transform만
train_data_scaled = scaler.fit_transform(train_data_select)
test_data_scaled = scaler.transform(test_data_select)

In [118]:
train_data_scaled=pd.DataFrame(train_data_scaled,columns=train_data_select.columns)
test_data_scaled=pd.DataFrame(test_data_scaled,columns=train_data_select.columns)

In [119]:
train_data_select

,과거보유일,성별,연령대,투자성향,자산구간,주거래상품군,Life_style,서비스등급,총투자기간,주거래업종구분,...,년도_잔고수량,년도_총투자기간,금액변화,매수횟수,고객별_종목업종,고객별_종목_hold,투자성형_자산구간,총투자제곱,주거래업종구분_ME,업종_주종목
0,6.0,1,9,3,2,2,9,5,5,8,...,3,1,38000.0,99,10,17.054015,6,25,19.437957,0
1,48.0,1,9,3,2,2,9,5,5,8,...,0,1,2490000.0,99,10,17.054015,6,25,19.437957,1
2,3.0,1,9,3,2,2,9,5,5,8,...,4,1,17600.0,99,10,17.054015,6,25,19.437957,0
3,13.0,1,9,3,2,2,9,5,5,8,...,2,1,7200.0,99,10,17.054015,6,25,19.437957,0
4,194.0,1,9,3,2,2,9,5,5,8,...,3,1,39400.0,99,10,17.054015,6,25,19.437957,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
681467,0.0,1,4,4,2,2,3,4,3,8,...,3,0,17250.0,162,10,17.054015,8,9,19.437957,0
681468,0.0,1,4,4,2,2,3,4,3,8,...,4,1,46550.0,162,10,17.054015,8,9,19.437957,0
681469,0.0,1,4,4,2,2,3,4,3,8,...,4,0,3665.0,162,10,17.054015,8,9,19.437957,0
681470,0.0,1,4,4,2,2,3,4,3,8,...,3,1,9050.0,162,10,17.054015,8,9,19.437957,0


In [120]:
test_data_select

,과거보유일,성별,연령대,투자성향,자산구간,주거래상품군,Life_style,서비스등급,총투자기간,주거래업종구분,...,년도_잔고수량,년도_총투자기간,금액변화,매수횟수,고객별_종목업종,고객별_종목_hold,투자성형_자산구간,총투자제곱,주거래업종구분_ME,업종_주종목
0,153,1,9,3,2,2,9,5,5,8,...,4,1,8150.0,99,10,17.054015,6,25,19.437957,0
1,335,1,9,3,2,2,9,5,5,8,...,3,1,12250.0,99,10,17.054015,6,25,19.437957,0
2,139,1,9,3,2,2,9,5,5,8,...,3,1,30600.0,99,10,17.054015,6,25,19.437957,0
3,236,1,9,3,2,2,9,5,5,8,...,4,1,1760.0,99,10,17.054015,6,25,19.437957,0
4,9,1,9,3,2,2,9,5,5,8,...,1,1,46350.0,99,10,17.054015,6,25,19.437957,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70591,23,1,6,0,5,2,5,3,5,2,...,0,1,198000.0,162,2,36.974653,0,25,22.077924,0
70592,103,1,6,0,5,2,5,3,5,2,...,2,1,2440.0,162,2,36.974653,0,25,22.077924,0
70593,858,1,4,4,2,2,3,4,3,8,...,4,0,-51.0,162,10,17.054015,8,9,19.437957,0
70594,11,1,4,4,2,2,3,4,3,8,...,2,1,116500.0,162,10,17.054015,8,9,19.437957,0


## Modeling-1
- test의 과거보유일로 train 의 과거보유일을 예측하는 모델 만들어서 값 넣는거 가능?

In [121]:
models = []

folds = KFold(n_splits=10)
for train_idx, val_idx in folds.split(train_data_scaled):
    
    train_x = train_data_scaled.iloc[train_idx, :]
    train_y = train_label[train_idx]
    val_x = train_data_scaled.iloc[val_idx, :]
    val_y = train_label[val_idx]
    
    model = LGBMRegressor(objective= "regression",
                          max_depth= 5,
                          n_estimators= 2000,
                          learning_rate= 0.01,
                          num_leaves = 31)
    
    model.fit(train_x, train_y,
              eval_set=[(val_x, val_y)],
              eval_metric=["rmse"],
              early_stopping_rounds=300,
              verbose=500)
    
    models.append(model)

Training until validation scores don't improve for 300 rounds
[500]	valid_0's rmse: 4.23665	valid_0's l2: 17.9492
[1000]	valid_0's rmse: 4.08469	valid_0's l2: 16.6847
[1500]	valid_0's rmse: 4.03135	valid_0's l2: 16.2517
[2000]	valid_0's rmse: 3.98429	valid_0's l2: 15.8745
Did not meet early stopping. Best iteration is:
[1999]	valid_0's rmse: 3.98423	valid_0's l2: 15.8741
Training until validation scores don't improve for 300 rounds
[500]	valid_0's rmse: 1.22162	valid_0's l2: 1.49236
[1000]	valid_0's rmse: 0.99383	valid_0's l2: 0.987699
[1500]	valid_0's rmse: 0.97213	valid_0's l2: 0.945037
[2000]	valid_0's rmse: 0.962793	valid_0's l2: 0.926971
Did not meet early stopping. Best iteration is:
[1995]	valid_0's rmse: 0.962327	valid_0's l2: 0.926072
Training until validation scores don't improve for 300 rounds
[500]	valid_0's rmse: 0.680721	valid_0's l2: 0.463381
[1000]	valid_0's rmse: 0.470222	valid_0's l2: 0.221108
Early stopping, best iteration is:
[1136]	valid_0's rmse: 0.469994	valid_0'

In [122]:
result = []
for i in models:
    result.append(i.predict(test_data_scaled))
predict = np.mean(result, axis = 0)

In [123]:
submission["hold_d"] = np.round(predict)

In [125]:
submission.to_csv("dacon_baseline.csv", index = False)

---

## Modeling

In [209]:
models = []

folds = KFold(n_splits=10)
for train_idx, val_idx in folds.split(train_data_scaled):
    
    train_x = train_data_scaled.iloc[train_idx, :]
    train_y = train_label[train_idx]
    val_x = train_data_scaled.iloc[val_idx, :]
    val_y = train_label[val_idx]
    
    model = LGBMRegressor(objective= "regression",
                          max_depth= 5,
                          n_estimators= 2000,
                          learning_rate= 0.01,
                          num_leaves = 31)
    
    model.fit(train_x, train_y,
              eval_set=[(val_x, val_y)],
              eval_metric=["rmse"],
              early_stopping_rounds=300,
              verbose=500)
    
    models.append(model)

Training until validation scores don't improve for 300 rounds
[500]	valid_0's rmse: 4.23773	valid_0's l2: 17.9583
[1000]	valid_0's rmse: 4.08812	valid_0's l2: 16.7127
[1500]	valid_0's rmse: 4.03805	valid_0's l2: 16.3058
[2000]	valid_0's rmse: 3.99292	valid_0's l2: 15.9434
Did not meet early stopping. Best iteration is:
[2000]	valid_0's rmse: 3.99292	valid_0's l2: 15.9434
Training until validation scores don't improve for 300 rounds
[500]	valid_0's rmse: 1.21832	valid_0's l2: 1.4843
[1000]	valid_0's rmse: 0.995689	valid_0's l2: 0.991397
Early stopping, best iteration is:
[835]	valid_0's rmse: 0.9818	valid_0's l2: 0.963931
Training until validation scores don't improve for 300 rounds
[500]	valid_0's rmse: 0.68334	valid_0's l2: 0.466954
[1000]	valid_0's rmse: 0.467854	valid_0's l2: 0.218888
Early stopping, best iteration is:
[1082]	valid_0's rmse: 0.467634	valid_0's l2: 0.218682
Training until validation scores don't improve for 300 rounds
[500]	valid_0's rmse: 0.572023	valid_0's l2: 0.32

In [210]:
pd.DataFrame(model.feature_importances_,index=train_data_select.columns).plot.bar()

In [211]:
result = []
for i in models:
    result.append(i.predict(test_data_scaled))
predict = np.mean(result, axis = 0)

In [212]:
submission["hold_d"] = np.round(predict)

In [213]:
submission.to_csv("dacon_baseline.csv", index = False)